In [ ]:
# any numbers: nwspol, netustm, yrbrn
# 6666 not applicable
# 7777 refusal
# 8888 don't know 
# 9999 no answer


# 0 - 40 years: eduyrs
# 66
# 77
# 88
# 99


# 0 - 10 (0 = fully reject, 10 = fully agree) 
# ppltrst, trstprl, trstlgl, trstplt, trstprt, trstep, trstun, stfeco, stfedu,euftf,atchctr
# 
# 66
# 77
# 88
# 99

# 1 - 4 (1 = very interested, 4 = not interested)
# polintr, 




# 1 - 5 (1 = not at all, 5 = a great deal)
# psppsgva, actorlga, cptppola, 


# 1 Male, 2 Female, 9 No answer: gndr, vote
# 3 not eligible, 7 refusal, 8 don't know

# 1 Yes 2 No 
# uemp12m, brncntr, ctzcntr, dscrgrp, facntr, mocntr
# 7 refusal, 8 don't know, 9 no answer

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import os

 ## Import Dataset

In [ ]:
ESS = pd.read_csv('ESS8e02_2.csv')
ESS

In [ ]:
ESS.dtypes

In [ ]:
# Which columns do I want to keep?

# nwspol       minutes spent watching/reading news about politics and current affairs
# netustm      internet use, how much time on typical day, in minutes
# ppltrst      most people can be trusted or you can't be too careful
# gndr         gender (1 Male - 2 Female) 
# yrbrn        year of birth
# intewde      place of interview: (1 East) or (2 West) Germany
# polintr      how interested in politics
# psppsgva     political system allows people to have a say in what the government does
# pspippla     
# cptppola     confident in own ability to participate in politics
# actrolga     able to take active role in political group
# trstprl      trust in country's parliament
# trstlgl      trust in the legal system
# trstplt      trust in politicians
# trstprt      trust in political parties
# trstep       trust in the European Parliament
# trstun       trust in the United Nations
# vote         voted in last election
# stfeco       how satisfied with present state of economy
# stfedu       state of education in country nowadays
# euftf        European unification should go further or has gone too far
# atchctr      how emotionally attached are you to your country
# dscrgrp      member of a group that is discriminated against in the country
# happy        how happy are you
# brncntr      born in country
# facntr       father born in country
# mocntr       mother born in country
# ctzcntr      citizen of country 
# eduyrs       years of full-time education completed
# uemp12m      any period of unemployment that lasted longer than 12 months
# edubde1      highest level of education successfully completed (school) -> Germany
# eduade2      highest level of education successfully completed (university etc) -> Germany

## Cleaning the dataset

In [ ]:
ESS.head()

In [ ]:
# pd.set_option('display.max_columns', None)

In [ ]:
ESS.info()

In [ ]:
ESS.shape

In [ ]:
# rename columns and put in lower case
# not necessary since the column names are already in lower case

# cols = []
# for colname in ESS.columns:
#     cols.append(colname.lower())
# ESS.columns = cols

# more renaming done in Notebook 1.04 (examples)

In [ ]:
ESS.dtypes
ESS

In [ ]:
ESS.isna().sum()

# ESS.isna() will only show False/True

In [ ]:
# tricky here:

# dataset barely has NaNs
# but answers like 'I dont know', 'No answer' is encoded as: 
# 6/66/666/6666  7/77/777/7777  8/88/888/8888
# different depending on column

In [ ]:
# what to do with these 'empty' values

# look at how large fraction of these values is 
# fill in the mean?

In [ ]:
# Drop columns before cleaning! Since there are too many to be cleaned otherwise

In [ ]:
# not done yet, insert more columns

data = ESS[['nwspol', 'netustm', 'ppltrst', 'gndr', 'yrbrn', 'agea', 'intewde', 'polintr', 'psppsgva', 'cptppola', 'actrolga', 'trstprl', 'trstlgl', 'trstplt', 'trstprt', 'trstep', 'trstun', 'vote', 'stfeco', 'stfedu', 'euftf', 'atchctr', 'atcherp', 'dscrgrp', 'happy', 'brncntr', 'facntr', 'mocntr', 'ctzcntr', 'eduyrs', 'uemp12m']]

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
# Choose only features that don't need to be recoded later (no nominal with >2 choices)
# Take features which would be used in draft model and categorize based on what miss dict to apply.


# 7, 8, 9 are missings 
features1 = ['polintr', 'psppsgva', 'actrolga', 'cptppola', 'vote', 'dscrgrp',
            'ctzcntr', 'brncntr', 'facntr', 'mocntr']


# 6, 7, 8, 9 are missings
features2 = ['gndr', 'uemp12m']

# 77, 88, 99 are missing
features3 = ['ppltrst', 'trstprl', 'trstlgl', 'trstplt', 'trstprt', 'trstep', 'trstun', 'stfeco', 'stfedu', 'euftf', 'happy', 'atchctr', 'atcherp',
             'eduyrs']

# 666, 777, 888, 999 are missings
features4 = ['agea']

# 7777, 8888, 9999 are missings
features5 = ['nwspol', 'yrbrn']

# 6666, 7777, 8888, 9999 are missings
features6 = ['netustm']

In [ ]:
all_vars = features1 + features2 + features3 + features4 + features5 + features6

In [ ]:
print("Overall number of variables:", len(all_vars))


In [ ]:
all_variables = all_vars

In [ ]:
# Dictionaries based on which the features numerical missings would be recoded to NaN
# One dictionary for each feature list (except features0 that don't contain any missings)

# Universal missing value
missing = pd.np.nan

# Dictionaries mapping numerics to missing var based on how features to recode were implemented. I check labels in SPSS.
missRecDict1 = {7: missing, 8: missing, 9: missing}
missRecDict2 = {6: missing, 7: missing, 8: missing, 9: missing}
missRecDict3 = {77: missing, 88: missing, 99: missing}
missRecDict4 = {666: missing, 777: missing, 888: missing, 999: missing}
missRecDict5 = {7777: missing, 8888: missing, 9999: missing}
missRecDict6 = {6666: missing, 7777: missing, 8888: missing, 9999: missing}


### Recoding missing from numericals into NaN

In [ ]:
# Save to Data_m, where 'm' stands for "missings".

Data_m = pd.DataFrame()
Data_m[features1] = data[features1].replace(missRecDict1)
Data_m[features2] = data[features2].replace(missRecDict2)
Data_m[features3] = data[features3].replace(missRecDict3)
Data_m[features4] = data[features4].replace(missRecDict4)
Data_m[features5] = data[features5].replace(missRecDict5)
Data_m[features6] = data[features6].replace(missRecDict6)


In [ ]:
# get the variables that are not country specific, ordinal, and not part of the admistrative group
ordinal = all_variables.query("Country_specific == \"no\" & Scale_type == \"ordinal\" and Group != \"Group Administrative variables\"")

# get the continous variables mentioned above 
continious = all_variables.query( "Name in [\"agea\",\"eduyrs\",\"nwspol\",\"netustm\"]")

# get the nominal variables mentioned above 
nominal = all_variables.query( "Name in [\"cntry\",\"gndr\"]")

# append them to one data frame
variables = pd.concat([nominal,continious,ordinal]).reset_index(drop=True)
variables.head(10)

In [ ]:
D

#### Save data with selected features and correctly encoded missings as Data_missings.csv

In [ ]:
Data_m.to_csv('Data_missings.csv',index=False)

## Exploratory Data Analysis

In [ ]:
# Which columns are numericals?
# Most must be categoricals
# How do I want to work with them

In [ ]:
# get the mean of different columns
#### EXAMPLE ######


In [ ]:
# Columns above threshold
# Lab 7.01 and 7.02 answers Drive Notebook


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
plt.scatter(eduyrs, trstprl)
plt.show()

In [ ]:
########## PUT IN BEGINNING CELL ##########################
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Barplots
sns.barplot(x="intewde", y="polintr", data=data)
plt.show()

# 1 = East Germany
# 2 = West Germany

In [ ]:
sns.barplot(x="intewde", y="trstprl", data=data)
plt.show()

In [ ]:
sns.barplot(x="intewde", y="trstep", data=data)
plt.show()

In [ ]:
sns.barplot(x="intewde", y="dscrgrp", data=data)

plt.show()

# 

In [ ]:
# Boxplots
sns.boxplot(x = 'eduyrs',y='trstlgl', data=data)
plt.show()

In [ ]:
# Distribution plots
sns.displot(data['eduyrs'], bins=60)
plt.show()

In [ ]:
#scatterplots are great to compare two distinct variables and see if they are somehow related!
sns.scatterplot(x=data['trstprt'], y=data['eduyrs'])
plt.xlim(0, 10)
plt.ylim(0, 60)
plt.show()

In [ ]:
# the big bomb of Visual Data Anlysis
sns.pairplot(data)
plt.show()

## replace the false values with mean!!
# https://stackoverflow.com/questions/48144828/how-to-replace-certain-values-in-a-pandas-column-with-the-mean-column-value-of-s

In [ ]:
# Any revelations through Data Visualization

In [ ]:
# Correlation: Is our datacorrelations_matrix = data.corr()
correlations_matrix = data.corr()
correlations_matrix

In [ ]:
# Heatmap
sns.heatmap(correlations_matrix, annot=True)
plt.show()

# looks crazy! replacement will probably change the look of the heatmap

In [ ]:
#############

# EVERYTHING CORRECT SO FAR? 
# HOW TO ANALYSE DATASET WITH ONLY CATEGORICALS? DIFFERENCE TO NUMERICALS?
# DISTRIBUTION AND EVERYTHING (PLOTS, ETC)

#######################

In [ ]:
### #### LINEAR REGRESSION MODEL #### ####

In [ ]:
# FIX LINEAR REGRESSION RE-DO COMPLETELY AFTER BOOTCAMP
# FOCUS ON PROPERLY ENCODING THE VARIABLES! DIFFICULT TO SEPERATE THE DIFFERENT VALUES AND MEANINGS OF VALUES
# FIND SMART SOLUTION

## X/y Split

In [ ]:
# Do X-y Split on dataset
y = data['TARGET_D']
X = data.drop(['TARGET_D'], axis=1)

# already encoded dataset
# TARGET (y) is the trust variable (choose different ones: trstprl, trstprt, trstep)

In [ ]:
X.head()

In [ ]:
# Categorical and numerical features are going to be treated differently

X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

## Normalizing numerical data

In [ ]:
# Normalizing and Standardizing data - only for numerical variables

In [ ]:
# we need scaling: in order to find out wheter we have to scale or not we look at the ranges
# range between min and max
# if the range is very big(in this example AVGGIFT after squaring in the 180000 and IC5 at 18 ~ unresonable)
X_num.describe().T

In [ ]:
# after MinMaxScaler you have to put X-normalized back into DataFrame, so that we can see the columns

from sklearn.preprocessing import MinMaxScaler
MinMaxtransformer = MinMaxScaler().fit(X_num)
X_normalized = MinMaxtransformer.transform(X_num)

print(X_normalized.shape)
X_normalized = pd.DataFrame(X_normalized,columns=X_num.columns)
X_normalized.head()

In [ ]:
X_normalized.describe().T

In [ ]:
from sklearn.preprocessing import StandardScaler
Standardtransformer = StandardScaler().fit(X_num)
X_standardized = Standardtransformer.transform(X_num)
print(X_standardized.shape)
X_standardized = pd.DataFrame(X_standardized,columns=X_num.columns)
X_standardized.head()

In [ ]:
X_standardized.describe().T

## Normalizing categorical data

In [ ]:
############ ENCODE CERTAIN CATEGORICAL COLUMNS DIFFERENTLY? #####################

In [ ]:
#one hot encoding is a way to turn categorical variables into multiple numerical columns
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder().fit(X_cat)
#endoder # ONeHotEncoder(drop='first').fit(X_cat)
print(encoder.categories_)
encoded = encoder.transform(X_cat).toarray()
print(encoded)
onehot_encoded = pd.DataFrame(encoded,columns=encoder.categories_)
onehot_encoded = pd.DataFrame(encoded,columns=['Female','Male','U'])
#onehot_encoded = pd.DataFrame(encoded,columns=[Male','U'])
onehot_encoded.head(20)

In [ ]:
# Concat all the information into a single dataset with all features
X = pd.concat([X_normalized, onehot_encoded], axis=1)

In [ ]:
X.head()

In [ ]:
y = data['TARGET_D']
y.head()

# Linear Regression

## Train-test-split

In [ ]:
# train test split is the way ML generates its claim to fame: 
# we build the model on a portion of the data but we then validate it in 
# another "fresh" portion
# our model has no opportunity to "cheat": it must accurately guess the values 
# in the "fresh" dataset that it never saw before
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#random=initial set of training is always different, that way I will always get the same rows and not different rows
#42 is a choice, its not given

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
#we train/fit our model like yesterday
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import r2_score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

In [ ]:
predictions = lm.predict(X_test)

In [ ]:
# But now we evaluate it in the TEST portion of the data, that we did not use for training.
# This way we know our model is genuinely guessing our donations, not just repeating the values it has seen in the training data


r2_score(y_test, predictions)

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,predictions))

# Model Validation - Metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# r2 score
predictions_test = lm.predict(X_test)
print("r2 Score of test Data: ",round(r2_score(y_test, predictions_test),3))
predictions_train = lm.predict(X_train)
print("r2 Score of train Data: ",round(r2_score(y_train, predictions_train),3), "\n")

# mse
y_pred = lm.predict(fifa_whole)
print("mean squared error: ",round(mean_squared_error(y_pred,y),3))

# rmse
print("rooted mean squared error: ", round(np.sqrt(mean_squared_error(y_pred,y)),3),"\n")

# mae
mae_test = mean_absolute_error(y_test, predictions_test)
print("mean absolute error of test data:",round(mae_test,3))
mae_train = mean_absolute_error(y_train, predictions_train)
print("mean absolute error of train data:", round(mae_train,3))

In [ ]:
mse=mean_squared_error(y_test,predictions_test)

In [ ]:
y_test.mean()

In [ ]:
predictions

In [ ]:
#we make the same separation into numerical and categorical
X_for_p_num = data_for_p.select_dtypes(include = np.number)
X_for_p_cat = data_for_p.select_dtypes(include = object)

In [ ]:
# for normalization and one hot encoding we need to make sure we remain consisten with the training data:
# If we MinMax scale just this piece of data, a "1" in this dataset means something very different from a 1 in the original dataset
# If we one-hot-encode in this dataset, the order of the columns (Unknown, Male, Female) may turn out different
#to avoid this, we use the same transformers we had already defined before, we do not fit them again.
encoded_for_p = encoder.transform(X_for_p_cat).toarray()
encoded_for_p
encoder.categories_
onehot_encoded_for_p = pd.DataFrame(encoded_for_p,columns=encoder.categories_)
#onehot_encoded_for_p = onehot_encoded_for_p.drop(['Female'],axis=1)
onehot_encoded_for_p.head()

In [ ]:
X_for_p_normalized = MinMaxtransformer.transform(X_for_p_num)
X_for_p_normalized = pd.DataFrame(X_for_p_normalized,columns=X_for_p_num.columns)

#merge back all of our labels
X_for_p = pd.concat([X_for_p_normalized, onehot_encoded_for_p], axis=1)

X_for_p.head()

In [ ]:
#predict and inspect results
results_for_p = lm.predict(X_for_p)

pd.concat([data_for_p,pd.Series(results_for_p, name='estimate')],axis=1).head()
#estimate is the result we get from p

In [ ]:
mae = mean_absolute_error(y_test, predictions)
print(mae)
#if there is a minus before, you'll get rid of it by taking the absolute number

In [ ]:
rmse = math.sqrt(mse)
print(rmse)

In [ ]:
r2 = r2_score(y_test, predictions)
r2

In [ ]:
############## LINEAR REGRESSION DONE? ####################

In [ ]:
############ RESULTS? WHAT INDICATIONS? HOW TO INTERPRET RESULTS? ####################

# Hypothesis Testing

In [ ]:
import scipy.stats as st
from scipy.stats import ttest_1samp

# p-value of 5%
pval = 0.05

In [ ]:
# Lab 4.4 for Help
# Hypothesis:
# HO:
# H1

In [ ]:
# two tailed test
# one tailed test
# crosstab function

In [ ]:
# Using the crosstab function, find the department that has the most hourly workers.
# department = pd.crosstab(salaries_df['Department'],salaries_df['Salary or Hourly']).sort_values(by='Hourly',ascending=False).reset_index()
# department

# Construction Confidence Intervals

In [ ]:
# Lab 4.4

## Hypothesis Tests for Proportions

In [ ]:
# Lab 4.4

In [ ]:
##### WHAT DO WE LEARN FROM MY ANALYSIS ########
######## GIVES REASON FOR FURTHER RESEARCH? #########
####### RELEVANT INFORMATION ##########
####### ANYTHING SURPRISING? DID IT CONFIRM MY ASSUMPtIONS #######
###### PUT ASSUMPTIONS IN THE INTRO #  AND HYPOTHESIS TESTING ###########